# Creating topics for the cases
This notebooks explores the topics we can create from the cases. It's gonna involve a lot of trial and error, but it will hopefully be well-documented

In [13]:
import pickle
import requests
import random
import numpy as np
from pathlib import Path
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
def read_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

def flatten_embeddings(embedding_dict):
    """ Creates a big matrix with all the embeddings from the dict """
    return np.vstack(embedding_dict.values())

def flatten_list(lst):
    return [elem for sublist in lst for elem in sublist]

def get_paragraphs(paragraph_dict):
    return flatten_list(list(paragraph_dict.values()))


def load_text_url(text_url: str): 
    """Inputs a URL of a newline seperated text-file and returns a list"""
    response = requests.get(text_url)
    return response.text.split("\n")

## Loading the data

In [10]:
DATA_DIR = Path("../../BscThesisData/data")
embedding_dict = read_pickle(DATA_DIR / "embedding_dict.pkl")
clean_paragraphs = read_pickle(DATA_DIR / "paragraph_dict.pkl")

Now it's time to extract the embeddings for BERTopic to process

In [14]:
embeddings = flatten_embeddings(embedding_dict)
docs = get_paragraphs(clean_paragraphs)
print(random.sample(docs, 5))

['Sagens ene spørgsmål var, om skatteyderen var berettiget til et nedslag i beregningsgrundlaget for rådighedsbeskatning af fri bolig, fordi der var indrettet hjemmekontor i en villa, som skatteyderen fik stillet til rådighed af et selskab, som han var hovedaktionær og direktør i.', 'Sagen angik, om de forhøjede parkeringsafgifter/kontrolafgifter, som appellanten, et parkeringsselskab, opkræver af bilister, der parkerer deres bil i strid med forskrifterne i privatejede parkeringsanlæg, som appellanten er operatør på, udgør vederlag for en momspligtig udlejning af parkeringsplads, jf. momslovens § 4, stk. 1, 1. pkt., jf. § 13, stk. 1, nr. 8, 2. pkt. Sagen angik 13 nærmere opregnede tilfælde, hvor appellanten opkræver sådanne forhøjede parkeringsafgifter eller kontrolafgifter.', 'Herefter fandt retten det ikke bevist, at boligselskabet skulle betale advokatselskabets honorar, hvorfor betingelsen i skatteforvaltningslovens § 52, stk. 1, ikke var opfyldt.', 'Med virkning fra d. 1. juli 201

### Now we initialize the models

# Load Cleaning Models

In [15]:
STOP_WORD_URL = "https://gist.githubusercontent.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b/raw/305d8e3930cc419e909d49d4b489c9773f75b2d6/stopord.txt"
STOP_WORDS = load_text_url(STOP_WORD_URL)
vectorizer_model = CountVectorizer(stop_words=STOP_WORDS)

In [25]:
# Powering up the transformer!
topic_model = BERTopic("Maltehb/-l-ctra-danish-electra-small-cased", vectorizer_model=vectorizer_model, nr_topics=5)

In [26]:
topics, probs = topic_model.fit_transform(docs, embeddings)

In [27]:
topic_model.visualize_topics()